In [25]:
import numpy as np
import pandas as pd
from scipy.stats import mode

In [12]:
#Initializ with centroid data and the corresponding uncertainty generated from Feature Extraction
STAR_CENTROIDS = np.array([[-3,4], [8,-6], [10,10], [5,5]]) 
STAR_CENTROIDS_UNCERTAINTY = np.array([[0.06], [0.004], [0.001], [0.002]])
NUM_STARS = STAR_CENTROIDS.shape[0]
#Initialize 
CATALOGUE = pd.read_csv("Modified Star Catalogue.csv")
PROCESSED_CATALOGUE= pd.read_csv('Processed_Catalogue.csv')

In [4]:
def cos(row):
    return np.cos(np.radians(row['Ang_Distance']))

REFERENCE  = pd.DataFrame(columns=['Star_ID1', 'Star_ID2', 'Ang_Distance'])
REFERENCE['Star_ID1'], REFERENCE['Star_ID2'] = PROCESSED_CATALOGUE['Star_ID1'], PROCESSED_CATALOGUE['Star_ID2']
REFERENCE['Ang_Distance'] = PROCESSED_CATALOGUE.apply(cos, axis = 1)
REFERENCE.sort_values('Ang_Distance' ,ascending=True, inplace=True)

REFERENCE.head()

,Star_ID1,Star_ID2,Ang_Distance
925,23440,80559,-0.999995
2832,23440,57018,-0.999955
2537,23440,23564,-0.999867
2502,23440,1403,-0.999534
2242,23440,68540,-0.999342


In [107]:
REF_ARR = REFERENCE.to_numpy()
REF_ARR

array([[ 2.34400000e+04,  8.05590000e+04, -9.99994623e-01],
       [ 2.34400000e+04,  5.70180000e+04, -9.99954887e-01],
       [ 2.34400000e+04,  2.35640000e+04, -9.99866872e-01],
       ...,
       [ 2.34400000e+04,  1.74600000e+03,  9.99125271e-01],
       [ 2.34400000e+04,  6.88640000e+04,  9.99393299e-01],
       [ 2.34400000e+04,  5.72240000e+04,  9.99698493e-01]])

In [6]:
STAR_CENTROIDS

array([[-3,  4],
       [ 8, -6],
       [10, 10],
       [ 5,  5]])

In [7]:
STAR_CENTROIDS_UNCERTAINTY

array([[0.06 ],
       [0.004],
       [0.001],
       [0.002]])

In [8]:
def starVectorTransform(centroid, focal_length=10):
    '''
    Generates the unit 3D vectors from given 2D centroids of stars on the 
    image frame with the focal point as the origin
    
    <Formula> - CubeStar Doc - Appendix B
    '''
    x, y = centroid
    
    temp = np.power(((x/focal_length)**2 + (y/focal_length)**2 + 1), -0.5)
    ux = (x/focal_length)
    uy = (y/focal_length)
    uz = 1
    return np.array([ux, uy, uz])*temp

STAR_VECTORS = np.apply_along_axis(starVectorTransform, 1, STAR_CENTROIDS, focal_length=10 )
STAR_VECTORS

array([[-0.26832816,  0.35777088,  0.89442719],
       [ 0.56568542, -0.42426407,  0.70710678],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.40824829,  0.40824829,  0.81649658]])

In [97]:
def vectorAngularDistance(vect1, vect2):
    '''
    Returns the angular distance [cos(theta)] between two unit vectors seperated by an angle theta
    '''
    return np.sum(vect1*vect2)

def uncertaintyAngularDistance(u1, u2):
    '''
    Assumes that the uncertainty is a simple addition
    '''
    return u1 + u2

In [121]:
temp = [[1]]
for i in range(NUM_STARS-1):
    temp.append([])
temp = np.array(temp)
temp[0].remove(1)

VOTE_LIST = np.vstack((np.arange(0, NUM_STARS), temp)).T
VOTE_LIST

array([[0, list([])],
       [1, list([])],
       [2, list([])],
       [3, list([])]], dtype=object)

In [122]:
for i in range(NUM_STARS):
    for j in range(i+1, NUM_STARS):
        d_ij = vectorAngularDistance(STAR_VECTORS[i], STAR_VECTORS[j])
        e_ij = uncertaintyAngularDistance(STAR_CENTROIDS_UNCERTAINTY[i], STAR_CENTROIDS_UNCERTAINTY[j])[0]
        r_ij = [d_ij - e_ij, d_ij + e_ij]
        
        ind = np.where( (REF_ARR[:, 2] >= r_ij[0]) & (REF_ARR[:,2] <= r_ij[1]) )
        for k in REF_ARR[ind]:
            s1, s2 = k[0], k[1]
            VOTE_LIST[i, 1].append(s1)
            VOTE_LIST[i, 1].append(s2)
            VOTE_LIST[j, 1].append(s1)
            VOTE_LIST[j, 1].append(s2)

In [220]:
temp = np.arange(0, NUM_STARS)
VOTE_LIST_2 = np.vstack((temp, np.zeros_like(temp),np.zeros_like(temp))).T
#VOTE_LIST_2[:, 2] = -1
VOTE_LIST_2

array([[0, 0, 0],
       [1, 0, 0],
       [2, 0, 0],
       [3, 0, 0]])

In [221]:
for i in range(NUM_STARS):
    VOTE_LIST_2[i,1] = mode(VOTE_LIST[i,1])[0][0]

In [222]:
VOTE_LIST_2

array([[    0, 23440,     0],
       [    1, 23440,     0],
       [    2, 23440,     0],
       [    3, 23440,     0]])

In [223]:
for i in range(NUM_STARS):
    for j in range(i+1, NUM_STARS):
        d_ij = vectorAngularDistance(STAR_VECTORS[i], STAR_VECTORS[j])
        e_ij = uncertaintyAngularDistance(STAR_CENTROIDS_UNCERTAINTY[i], STAR_CENTROIDS_UNCERTAINTY[j])[0]
        r_ij = [d_ij - e_ij, d_ij + e_ij]
        s1, s2 = VOTE_LIST_2[i, 1], VOTE_LIST_2[j, 1]
        
        ind1 = np.where( (REF_ARR[:, 0] == s1) & (REF_ARR[:,1] == s2) )        
        if ind1[0].shape != (0,):
            if REF_ARR[ind1]>r_ij[0] and REF_ARR[ind1]<r_ij[1]:
                VOTE_LIST_2[i,2] +=1
                VOTE_LIST_2[j,2] +=1
            continue
        
        ind2 = np.where( (REF_ARR[:, 0] == s2) & (REF_ARR[:,1] == s1) )
        if ind2[0].shape != (0,):
            if REF_ARR[ind2]>r_ij[0] and REF_ARR[ind2]<r_ij[1]:
                VOTE_LIST_2[i,2] +=1
                VOTE_LIST_2[j,2] +=1

In [224]:
VOTE_LIST_2

array([[    0, 23440,     0],
       [    1, 23440,     0],
       [    2, 23440,     0],
       [    3, 23440,     0]])